# Validation of solutions for models with Master polytropic EoS
## Varying politropic index $n$ and $\sigma \ (= \frac{P_{c}}{\rho_{c}})$

This notebook shows the comparison between dimensionless radii and masses at the surface ($\xi_{b},\eta_b$) for the master polytropic EoS with $\alpha = \beta = 0$, against dimensionless radii and masses ($\bar{\xi}_b, \bar{\eta}_b$) obtained in [T. Dallas y V. Geroyannis](http://adsabs.harvard.edu/pdf/1993Ap%26SS.201..249D).

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

# Compact object modeling

## Structure equations

The structure equations are the result of manipulating the Einstein field equations for a given metric and energy tensor. They are the equations to be solved to model compact objects. In the case of static conigurations, with spherical symmetry and anisotropic pressure, the structure equations consist of
\begin{eqnarray}
P^{\prime} &=& - \left(\rho + P \right) \frac{m + 4 \pi r^{3} P}{r(r - 2m)} + \frac{2}{r} \left(P_{\perp} - P \right)  \label{EqHid} \tag{1} \\
m^{\prime} &=& 4 \pi r^{2} \rho \label{MasDif} \tag{2}
\end{eqnarray}
where $\rho$ is energy density, $m$ is mass, $P$ is radial pressure and $P_{\perp}$ is tangential pressure. The prime denotes derivative with respect to $r$.

## Equation of State

An equation of state (EoS) is a mathematical model between state variables that describes the most important physical processes that occur in a thermodynamic system. The master polytropic EoS consists of a relationship such that
\begin{equation}
P = \kappa \rho^{1+\frac{1}{n}} + \alpha \rho - \beta \,. \label{PoliMaestra} \tag{3}
\end{equation}

On the other hand, the anisotropic pressure is supposed to be of the form
\begin{equation}
\Delta\equiv P_{\perp} - P = C r (\rho + P) \frac{m + 4 \pi r^3 P}{r(r-2m)} \,, \label{Anisotropia} \tag{4}
\end{equation}
such that equation $\eqref{EqHid}$ is as
\begin{equation}
\frac{\mathrm{d}P}{\mathrm{d}r} = - h \frac{(\rho + P)(m + 4 \pi  r^3 P)}{r(r-2m)} \,, \label{EqHidCos} \tag{5}
\end{equation}
where $h = 1 - 2C$, and $C$ quantifies the anisotropy in the model.

## Lane-Emden equation

The structure equations can be written dimensionless when they are endowed with polytropic EoS. The result is known as the Lane-Emden equation, given by the change of variables
\begin{equation}
\Psi^{n}(\xi) = \frac{\rho}{\rho_{c}} \ , \ \ \eta \left(\xi \right) = \frac{m}{4 \pi \rho_c a^{3}} \quad \textrm{and} \quad r = a\xi \,,
\end{equation}
where
\begin{equation}
a^{2} = \frac{\Upsilon \left(n + 1 \right)}{4 \pi \rho_c} \ , \ \ \Upsilon = \kappa \rho_{c}^{1/n} = \frac{\sigma - \alpha \left(1 - \varkappa \right)}{1 - \varkappa^{1 + \frac{1}{n}}} \ , \ \ \sigma = \frac{P_{c}}{\rho_{c}} \quad \textrm{and} \quad \varkappa = \frac{\rho_{b}}{\rho_{c}} \,.
\end{equation}
The subscripts $c$ and $b$ indicate that the variable is evaluated at the center and surface of the configuration, respectively.

In this way, the dimensionless EoS $\eqref{PoliMaestra}$ and $\eqref{Anisotropia}$ are
\begin{eqnarray}
P &=& \rho_c \left\{\Upsilon \left( \Psi^{n+1} - \varkappa^{1 + \frac{1}{n}} \right) + \alpha \left(\Psi^{n} - \varkappa \right)\right\}  = \rho_c \mathcal{P} \quad \textrm{and} \label{PAdi} \tag{6} \\
\Delta &=& \frac{C \Upsilon (n+1) \left(\eta + \xi^{3} \mathcal{P} \right) \left(\Psi^{n} + \mathcal{P} \right) \rho_{c}}{\xi - 2  \Upsilon \left( n+1 \right) \eta} \,, \label{AniAdi} \tag{7}
\end{eqnarray}
respectively.

Finally, equations $\eqref{EqHidCos}$ and $\eqref{MasDif}$, written in dimensionless form, are
\begin{eqnarray}
\dot{\Psi} &=& - \frac{h \left(\eta + \xi^{3} \mathcal{P}\right) \left(1 + \frac{\mathcal{P}}{\Psi^{n}}\right)}{\xi \left\{\xi-2\,\Upsilon\,\left( n+1 \right) \eta\right\} \left\{1 + \frac{\alpha n}{\Upsilon \left(n+1\right) \Psi}\right\}}  \qquad \textrm{and} \label{PsiPunto} \tag{8} \\
\dot{\eta} &=& \xi^{2}\Psi^{n} \,, \label{EtaPunto} \tag{9}
\end{eqnarray}
where dot indicates derivative with respect to $\xi$.

Therefore, the system of equations to integrate numerically is given by $\eqref{PsiPunto}$ and $\eqref{EtaPunto}$, with initial conditions
\begin{equation}
\Psi (\xi = 0) = \Psi_{c} = 1 \,, \quad \eta (\xi = 0) = \eta_{c} = 0  \,,
\end{equation}
and boundary condition
\begin{equation}
\Psi (\xi = \xi_{b}) = 0 \,.
\end{equation}

In [3]:
# Defining system of equations: derivative of Psi and derivative of Eta as a function of xi
def funciones(xi,y, alpha, n, h, Upsilon, varkappa):
    psi_ , eta_ = y
    dydxi = [-h*(eta_ + xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1+1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - (varkappa**(1+1/n)/psi_**(n))) + alpha*(1 - (varkappa/psi_**(n))))/xi/(xi - 2*Upsilon*(n+1)*eta_) /(1 + alpha*n/Upsilon/(n+1)/psi_) 
             ,xi**(2)*psi_**(n)] 
    return dydxi

In [1]:
# The input parameters that characterize each model are defined: n, C, alpha, varkappa, sigma

Lista_n = [0.5,0.99999,1.5,2.0,2.5,2.99999,3.5,4.0]      # List of polytropic indices n
print('Values for n: ',end='')                                                    
print(*Lista_n, sep=', ')

C = 0.0  # Anisotropic factor value

h = 1 - 2*C

alpha = 0.0 # Linear term value

varkappa = 0.0 # Ratio between surface density and central density

# Sigma: Ratio between central pressure and central density
Lista_sigma = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]      # List of sigmas
print('Values for \u03c3: ',end='')                                                    
print(*Lista_sigma, sep=', ')

Values for n: 0.5, 0.99999, 1.5, 2.0, 2.5, 2.99999, 3.5, 4.0
Values for σ: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9


In [6]:
Listaxi1 = []    # Container of the values for xi on the surface
ListaEta1 = []   # Container of the values for eta on the surface

In [7]:
for i in range(len(Lista_n)):
    
    n = Lista_n[i]   # Polytropic index n for each model
    
    for j in range(len(Lista_sigma)):
        
        sigma = Lista_sigma[j]  # sigma for each model
        
       # Upsilon calculation
        Upsilon = (sigma - alpha*(1 - varkappa))/(1 - varkappa**(1 + 1/n)) 
        
        # Initial conditions
        Psi0 = 1.0
        Eta0 = 0.0
        y0 = [Psi0,Eta0]
        
        # Integration interval
        xi0 = 10**(-15)     # Start
        ximax = 10**(11)    # End, in case the condition to stop integration is not fulfilled
        xi_span = (xi0,ximax) # Integration space
    
        # Condition to stop integration (Dimensionless pressure less than 10**(-15))
        def stop_condition(xi,y,alpha, n, h, Upsilon, varkappa):
            return (1/sigma)*(Upsilon*(y[0]**(n+1) - varkappa**(1+1/n)) + alpha*(y[0]**(n) - varkappa)) - 10**-15
        stop_condition.terminal = True
    
        # Solution of the system of equations using the routine "solve_ivp" by means of the RK45 method
        # solve_ivp(Equations to integrate, Integration space, Initial conditions, Integration method,
        #           Condition to stop integration)
        soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                               args=(alpha, n, h, Upsilon, varkappa))
        xi = soluciones.t
        Psi = soluciones.y[0]
        Eta = soluciones.y[1]
        
        Listaxi1.append(np.around(xi[-1],7))
        ListaEta1.append(np.around(Eta[-1],7))

C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


Wall time: 1.1 s


In [8]:
pd.options.display.float_format = "{:,.7f}".format

# Reading of data taken from T. Dallas and V. Geroyannis, “The boundary conditions for relativistic polytropic fluid spheres,
# ”Astrophysics and space science, vol. 201, no. 2, pp. 249–271,1993.
data = pd.read_csv('Validación - Datos Dallas y Geroyannis (1993)',names = ['N', 'Sigma', 'Xi_b', 'Eta_b'],
                  engine='python')

In [9]:
data['Xib'] = Listaxi1      # Add the solutions obtained, for xi at the surface, to the table
data['Etab'] = ListaEta1    # Add the solutions obtained, for eta at the surface, to the table

errorXi = (abs(data['Xi_b'] - data['Xib'])/data['Xi_b'])*100      # Calculation of the percentage error for xi at the surface
errorEta = (abs(data['Eta_b'] - data['Etab'])/data['Eta_b'])*100  # Calculation of the percentage error for eta at the surface
data['Error Xi_b'] = errorXi            # Add the error, for xi at the surface, to the table
data['Error Eta_b'] = errorEta          # Add the error, for xi at the surface, to the table

pd.set_option("display.max_rows", None, "display.max_columns", None)
data

,N,Sigma,Xi_b,Eta_b,Xib,Etab,Error Xi_b,Error Eta_b
0,0.5000000,0.1000000,2.2899149,2.1743665,2.2894217,2.1745060,0.0215379,0.0064157
1,0.5000000,0.2000000,2.0008463,1.4374281,2.0000929,1.4377010,0.0376541,0.0189853
2,0.5000000,0.3000000,1.8013545,1.0360530,1.8009694,1.0360952,0.0213784,0.0040732
3,0.5000000,0.4000000,1.6543434,0.7912060,1.6540702,0.7912125,0.0165141,0.0008228
4,0.5000000,0.5000000,1.5408662,0.6296190,1.5406534,0.6296304,0.0138104,0.0018043
5,0.5000000,0.6000000,1.4502055,0.5166507,1.4500638,0.5166762,0.0097710,0.0049356
6,0.5000000,0.7000000,1.3758251,0.4341211,1.3756575,0.4341512,0.0121818,0.0069289
7,0.5000000,0.8000000,1.3135016,0.3717053,1.3133043,0.3717356,0.0150209,0.0081543
8,0.5000000,0.9000000,1.2603785,0.3231648,1.2601465,0.3231968,0.0184072,0.0099052
9,1.0000000,0.1000000,2.5990904,1.7514303,2.5990779,1.7516917,0.0004809,0.0149249
